In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm.notebook as tqdm

import gen_params as gp

In [2]:
# Set up directories
cwd = os.getcwd()
if 'dev' in cwd:
    parent_dir = os.path.dirname(cwd)
    data_dir = os.path.join(parent_dir, 'data')
else:
    data_dir = os.path.join(cwd, 'data')

grid_data_dir = os.path.join(data_dir, 'grid', '2018Baseline')
if not os.path.exists(grid_data_dir):
    raise FileNotFoundError('Grid data directory not found.')

thermal_data_dir = os.path.join(data_dir, 'thermal')
if not os.path.exists(thermal_data_dir):
    raise FileNotFoundError('Thermal data directory not found.')

print('Grid data directory: {}'.format(grid_data_dir))
print('Thermal data directory: {}'.format(thermal_data_dir))

Grid data directory: /mnt/Bo_HDD/NYgrid-python/data/grid/2018Baseline
Thermal data directory: /mnt/Bo_HDD/NYgrid-python/data/thermal


# Read generator parameters

In [3]:
# Read processed generator parameters
gen_params_df = pd.read_excel(os.path.join(thermal_data_dir, 'gen_params_2018.xlsx'), index_col=0)
gen_params_df_2 = pd.read_excel(os.path.join(thermal_data_dir, 'gen_params_2024.xlsx'), index_col=0)
gen_params_df = pd.concat([gen_params_df, gen_params_df_2], axis=0)

In [17]:
# 2018 NYCA: 236 thermal generators
gen_combiner_2018 = pd.read_excel(os.path.join(thermal_data_dir, '2018_nyca_thermal.xlsx'),
                                  sheet_name='matched_with_id_filtered')
gen_combiner_2018 = gen_combiner_2018.rename(columns={
    '   Station        Unit': 'NYISO_Name'})

# Map unit type GT, JE to CT
gen_combiner_2018['Unit Type'] = gen_combiner_2018['Unit Type'].replace(
    {'GT': 'CT', 'JE': 'CT'})
gen_combiner_2018['Fuel Type Secondary'] = gen_combiner_2018['Fuel Type Secondary'].astype(
    str).replace({'nan': ''})
gen_combiner_2018 = gen_combiner_2018.set_index('ID')
gen_combiner_2018 = gen_combiner_2018.sort_values(
    by=['CAMD_Plant_ID', 'CAMD_Unit_ID'])

gen_params_2018 = gen_params_df.loc[gen_combiner_2018.index]
gen_params_2018 = gen_params_2018.join(gen_combiner_2018['gen_bus'])
gen_params_2018 = gen_params_2018[[
    'NYISO_Name', 'PTID', 'Unit_Type', 'Fuel_Type_Primary',
    'Fuel_Type_Secondary', 'CAMD_Plant_ID', 'CAMD_Unit_ID',
    'raise_rr_ratio', 'lower_rr_ratio',
    'max_gen', 'eco_min', 'eco_min_ratio',
    'heat_1', 'heat_0', 'gen_bus'
]]

In [18]:
gen_params_2018

,NYISO_Name,PTID,Unit_Type,Fuel_Type_Primary,Fuel_Type_Secondary,CAMD_Plant_ID,CAMD_Unit_ID,raise_rr_ratio,lower_rr_ratio,max_gen,eco_min,eco_min_ratio,heat_1,heat_0,gen_bus
ID,,,,,,,,,,,,,,,
"(2480,1)",Danskammer 1,23586,ST,NG,FO6,2480,1,0.500000,0.621622,74,17.76,0.24,13.132953,1.503283e+02,77
"(2480,2)",Danskammer 2,23589,ST,NG,FO6,2480,2,0.394366,0.507042,71,15.62,0.22,11.314618,6.529064e+01,77
"(2480,3)",Danskammer 3,23590,ST,NG,NaN,2480,3,0.382550,0.503356,149,53.64,0.36,5.579322,5.564137e+02,77
"(2480,4)",Danskammer 4,23591,ST,NG,NaN,2480,4,0.350211,0.506329,237,66.36,0.28,8.069488,8.318776e+01,77
"(2490,20)",Arthur Kill ST 2,23512,ST,NG,NaN,2490,20,0.421488,0.876033,363,94.38,0.26,8.503720,2.534583e+02,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(56964,GT4)",Bayonne EC CTG4,323685,CT,NG,KER,56964,GT4,0.937500,0.359375,64,32.00,0.50,9.007330,4.892416e-14,81
"(56964,GT5)",Bayonne EC CTG5,323686,CT,NG,KER,56964,GT5,0.890625,0.359375,64,32.00,0.50,9.112635,-6.050059e-15,81
"(56964,GT6)",Bayonne EC CTG6,323687,CT,NG,KER,56964,GT6,0.937500,0.375000,64,32.00,0.50,8.816724,2.240418e-13,81


In [15]:
gen_params_2018.columns

Index(['NYISO_Name', 'PTID', 'Unit_Type', 'Fuel_Type_Primary',
       'Fuel_Type_Secondary', 'CAMD_Facility_Name', 'CAMD_Plant_ID',
       'CAMD_Unit_ID', 'CAMD_Nameplate_Capacity', 'raise_rr_ratio',
       'lower_rr_ratio', 'max_gen', 'eco_min', 'eco_min_ratio', 'heat_1',
       'heat_0', 'gen_bus'],
      dtype='object')

In [5]:
# 2024 NYCA: 218 thermal generators
gen_combiner_2024 = pd.read_excel(os.path.join(thermal_data_dir, '2024_nyca_thermal.xlsx'),
                             sheet_name='matched_with_id_filtered')
gen_combiner_2024 = gen_combiner_2024.rename(columns={
    '   Station        Unit': 'NYISO_Name'})

# Map unit type GT, JE to CT
gen_combiner_2024['Unit Type'] = gen_combiner_2024['Unit Type'].replace(
    {'GT': 'CT', 'JE': 'CT'})
gen_combiner_2024['Fuel Type Secondary'] = gen_combiner_2024['Fuel Type Secondary'].astype(
    str).replace({'nan': ''})
gen_combiner_2024 = gen_combiner_2024.set_index('ID')
gen_combiner_2024 = gen_combiner_2024.sort_values(by=['CAMD_Plant_ID', 'CAMD_Unit_ID'])

gen_params_2024 = gen_params_df.loc[gen_combiner_2024.index]

# Format generator properties

In [8]:
gen_params_2018.columns

Index(['NYISO_Name', 'PTID', 'Unit_Type', 'Fuel_Type_Primary',
       'Fuel_Type_Secondary', 'CAMD_Facility_Name', 'CAMD_Plant_ID',
       'CAMD_Unit_ID', 'CAMD_Nameplate_Capacity', 'raise_rr_ratio',
       'lower_rr_ratio', 'max_gen', 'eco_min', 'eco_min_ratio', 'gen_sum',
       'heat_sum', 'heat_1', 'heat_0', 'heat_r2', 'nox_1', 'nox_0', 'nox_r2',
       'nox_sum', 'co2_1', 'co2_0', 'co2_r2', 'co2_sum', 'so2_1', 'so2_0',
       'so2_r2', 'so2_sum'],
      dtype='object')

In [7]:
gen_params_2018

,NYISO_Name,PTID,Unit_Type,Fuel_Type_Primary,Fuel_Type_Secondary,CAMD_Facility_Name,CAMD_Plant_ID,CAMD_Unit_ID,CAMD_Nameplate_Capacity,raise_rr_ratio,...,nox_r2,nox_sum,co2_1,co2_0,co2_r2,co2_sum,so2_1,so2_0,so2_r2,so2_sum
ID,,,,,,,,,,,,,,,,,,,,,
"(2480,1)",Danskammer 1,23586,ST,NG,FO6,Danskammer Generating Station,2480,1,72.0,0.500000,...,0.642841,3862.727,0.062075,-0.721211,0.932970,2981.415,0.103852,-10.167798,0.016332,NaN
"(2480,2)",Danskammer 2,23589,ST,NG,FO6,Danskammer Generating Station,2480,2,73.5,0.394366,...,0.346474,4907.791,0.059314,-0.019229,0.999995,2231.469,0.000580,0.008430,0.961099,NaN
"(2480,3)",Danskammer 3,23590,ST,NG,NaN,Danskammer Generating Station,2480,3,147.1,0.382550,...,0.448012,41331.583,0.059266,0.009782,0.999995,12956.599,0.000610,-0.004109,0.972907,NaN
"(2480,4)",Danskammer 4,23591,ST,NG,NaN,Danskammer Generating Station,2480,4,239.4,0.350211,...,0.882496,9269.651,0.059280,-0.003171,0.999999,6285.537,0.000588,0.015797,0.994084,NaN
"(2490,20)",Arthur Kill ST 2,23512,ST,NG,NaN,Arthur Kill,2490,20,376.2,0.421488,...,0.810997,340423.518,0.059430,-0.002278,0.999999,319606.767,0.000600,0.000045,0.999999,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(56964,GT4)",Bayonne EC CTG4,323685,CT,NG,KER,Bayonne Energy Center,56964,GT4,64.0,0.937500,...,0.908469,4933.457,0.059430,0.000172,0.999994,45990.386,0.000548,0.009155,0.923160,440.691
"(56964,GT5)",Bayonne EC CTG5,323686,CT,NG,KER,Bayonne Energy Center,56964,GT5,64.0,0.890625,...,0.867139,2597.524,0.059429,-0.000306,0.999995,22484.508,0.000573,0.004575,0.946402,221.289
"(56964,GT6)",Bayonne EC CTG6,323687,CT,NG,KER,Bayonne Energy Center,56964,GT6,64.0,0.937500,...,0.856754,2503.273,0.059429,0.000280,0.999996,22829.641,0.000559,0.005960,0.961417,220.448
